In [1]:
from thorlabs_apt_device.devices.bsc import BSC, BSC201 # https://thorlabs-apt-device.readthedocs.io/en/latest/

In [2]:
import serial
from serial.tools.list_ports import comports

In [3]:
import time

In [4]:
for i, p in enumerate(comports()):
    print(f'[{i}] {p}')

[0] /dev/ttyS0 - ttyS0
[1] /dev/ttyUSB2 - APT Stepper Motor Controller - APT Stepper Motor Controller
[2] /dev/ttyUSB3 - FT230X Basic UART - FT230X Basic UART


In [ ]:
# Sélection manuelle
appareil_apt = comports()[int(input('APT: '))]

In [5]:
# Sélection automatique
motif_apt = 'APT Stepper Motor Controller'

appareil_apt = next(filter(lambda x: motif_apt in str(x), comports()))

In [6]:
numéro_série = "40219394" # Pour l'appareil spécifique qu'on utilise

# Essai avec la classe modifié

In [12]:
dir(BSC201)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_atexit',
 '_close_port',
 '_process_message',
 '_run_eventloop',
 '_schedule_keepalives',
 '_schedule_reads',
 '_schedule_updates',
 '_write',
 'close',
 'home',
 'identify',
 'move_absolute',
 'move_jog',
 'move_relative',
 'move_velocity',
 'register_error_callback',
 'set_enabled',
 'set_home_params',
 'set_jog_params',
 'set_loop_params',
 'set_move_params',
 'set_trigger',
 'set_velocity_params',
 'stop',
 'unregister_error_callback']

In [67]:
# Basé sur https://thorlabs-apt-device.readthedocs.io/en/latest/api/thorlabs_apt_device.devices.bsc.html#thorlabs_apt_device.devices.bsc.BSC201
class BSC201_HDR50(BSC201):
    # Données tirées de Kinesis
    kinesis = {'travel [pas]': 27_033_577,
               'vel [pas/s]': 201_576_964,
               'acc [pas/s²]': 20_653,
               'jog_step [pas]': 37_547,
               'jog_vel [pas/s]': 40_315_393,
               'jog_acc [pas/s²]': 4_131,
               'travel [°]': 360,
               'vel [°/s]': 50,
               'acc [°/s²]': 25,
               'jog_step [°]': 0.5,
               'jog_vel [°/s]': 10,
               'jog_acc [°/s²]': 5}
    
    # Données tirées de https://www.thorlabs.com/drawings/8d6029bfe62e7acd-DF270E33-CE4C-A18B-8D52A70104946BCD/HDR50-Manual.pdf
    manuel = {'travel [°]': 360,
              'vel [°/s]': 50,
              'acc [°/s²]': 80,
              'macropas [°]': 1.8,
              'micropas [1/macropas]': 2048,
              'jog_step [pas]': kinesis['jog_step [pas]']}
    manuel['micropas [°]'] = manuel['macropas [°]'] / manuel['micropas [1/macropas]']
    manuel['travel [pas]'] = int( manuel['travel [°]'] / manuel['micropas [°]'] )
    manuel['vel [pas/s]'] = int( manuel['vel [°/s]'] / manuel['micropas [°]'] )
    manuel['acc [pas/s²]'] = int( manuel['acc [°/s²]'] / manuel['micropas [°]'] )
    
    def __init__(self,
                 serial_port=None,
                 vid=None,
                 pid=None,
                 manufacturer=None,
                 product=None,
                 serial_number='40',
                 location=None,
                 home=True,
                 invert_direction_logic=False,
                 swap_limit_switches=True,
                 réglages=kinesis):
        début = time.time()
        super().__init__(serial_port=serial_port,
                         vid=vid, pid=pid,
                         manufacturer=manufacturer,
                         product=product,
                         serial_number=serial_number,
                         location=location,
                         home=home,
                         invert_direction_logic=invert_direction_logic,
                         swap_limit_switches=swap_limit_switches)
    
        # Initial velocity parameters are effectively zero on startup, set something more sensible
        # Homing is initiated 1.0s after init, so hopefully these will take effect before then...
        # Tiré de la source pour BSC201_DRV250
        for bay_i, _ in enumerate(self.bays):
            for channel_i, _ in enumerate(self.channels):
                self.set_home_params(velocity=réglages['vel [pas/s]'],
                                     offset_distance=0,
                                     bay=bay_i,
                                     channel=channel_i)
                self.set_velocity_params(acceleration=réglages['acc [pas/s²]'],
                                         max_velocity=réglages['vel [pas/s]'],
                                         bay=bay_i,
                                         channel=channel_i)
                self.set_jog_params(size=réglages['jog_step [pas]'],
                                    acceleration=réglages['jog_acc [pas/s²]'],
                                    max_velocity=réglages['jog_vel [pas/s]'],
                                    bay=bay_i,
                                    channel=channel_i)
                # Use open-loop positioning (only using stepper counts)
                #self.set_loop_params(loop_mode=2, prop=50000, integral=5000, diff=100, pid_clip=16000000, pid_tol=80, encoder_const=4292282941, bay=bay_i, channel=channel_i)
                self.set_loop_params(loop_mode=1, prop=0, integral=0, diff=0, pid_clip=0, pid_tol=0, encoder_const=0, bay=bay_i, channel=channel_i)
        
        délai = time.time() - début
        if délai > 1:
            print(f"L'initialisation s'est faite trop lentement ({délai=}s).")


In [58]:
stage.close()

In [75]:
stage = BSC201_HDR50()
stage.identify() # La lumière sur le contrôleur externe devrait clignoter.

In [18]:
stage.genmoveparams

{'backlash_distance': 0,
 'msg': 'mot_get_genmoveparams',
 'msgid': 1084,
 'source': 33,
 'dest': 1,
 'chan_ident': 1}

In [19]:
stage.homeparams

{'home_dir': 2,
 'limit_switch': 1,
 'home_velocity': 201576964,
 'offset_distance': 0,
 'msg': 'mot_get_homeparams',
 'msgid': 1090,
 'dest': 1,
 'source': 33,
 'chan_ident': 1}

In [20]:
stage.jogparams

{'jog_mode': 2,
 'step_size': 37547,
 'min_velocity': 0,
 'acceleration': 4131,
 'max_velocity': 40315393,
 'stop_mode': 2,
 'msg': 'mot_get_jogparams',
 'msgid': 1048,
 'source': 33,
 'dest': 1,
 'chan_ident': 1}

In [21]:
stage.velparams

{'min_velocity': 0,
 'max_velocity': 201576964,
 'acceleration': 20653,
 'msg': 'mot_get_velparams',
 'msgid': 1045,
 'source': 33,
 'dest': 1,
 'chan_ident': 1}

In [76]:
stage.status

{'position': 0,
 'enc_count': 0,
 'velocity': 0.0,
 'forward_limit_switch': True,
 'reverse_limit_switch': False,
 'moving_forward': True,
 'moving_reverse': False,
 'jogging_forward': False,
 'jogging_reverse': False,
 'motor_connected': False,
 'homing': True,
 'homed': False,
 'tracking': True,
 'interlock': False,
 'settled': False,
 'motion_error': False,
 'motor_current_limit_reached': False,
 'channel_enabled': True,
 'msg': 'mot_get_statusupdate',
 'msgid': 1153,
 'source': 33,
 'dest': 1,
 'chan_ident': 1,
 'forward_limit_soft': False,
 'reverse_limit_soft': False,
 'initializing': False,
 'instrument_error': False,
 'overtemp': False,
 'voltage_fault': False,
 'commutation_error': False,
 'digital_in_1': True,
 'digital_in_2': False,
 'digital_in_3': True,
 'digital_in_4': False,
 'encoder_fault': False,
 'overcurrent': False,
 'current_fault': False,
 'power_ok': False,
 'active': False,
 'error': False}

In [69]:
stage.set_enabled(False)

In [70]:
stage.set_enabled(True)

In [62]:
stage.home()

In [64]:
stage.move_velocity(direction='forward')

In [33]:
stage.move_velocity(direction='reverse')

In [71]:
stage.move_absolute(int( BSC201_HDR50.kinesis['travel [pas]'] // 4 ))

In [72]:
stage.move_absolute(-int( BSC201_HDR50.kinesis['travel [pas]'] // 4 ))

In [39]:
stage.move_relative(int( BSC201_HDR50.kinesis['travel [pas]'] // 4 ))

In [41]:
stage.move_relative(int( BSC201_HDR50.kinesis['travel [pas]'] // 4 ))

In [77]:
stage.stop()

In [78]:
stage.close()